# Recipe Bot -- "What should I cook tonight?"

#### a simple rules-based chatbot to solve a common daily challenge most people face. written in Python.

```
Joke - 
"There are three main difficulties in life --
 What should I eat for breakfast?
 What to eat for lunch?
 What shall I have for a good dinner? "
```

### Step 1 Find a recipe database/dataset

For local computer experiment, let's just download a file containing enough recipes for search query. Can be replaced with a SQL database later.

I found only one recipe dataset on Kaggle. Link: https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions

It was used in a 2019 NLP paper by scholars from Hong Kong. (After I came to study in Canada, I'm often amazed that actually I didn't know how good HK scholars are in the research world.)

### Step 2 now load the recipe data for chatbot use

In [5]:
import pandas as pd

In [6]:
recipe_path = r"E:\recipe_bot\food_recipes\RAW_recipes.csv"
recipe_df = pd.read_csv(recipe_path)
recipe_df.head()
# file is 280Mb big, better just load the first few rows to check column format

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [7]:
#ingredients is what I mainly need for the chatbot
recipe_df['ingredients'][0]

"['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']"

In [8]:
#check steps format
recipe_df['steps'][0]

"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']"

In [9]:
#I need only 5 columns -- recipe name, recipe ingredients, recipe steps, description, submitted for now. Make a smaller df to ensure faster query time for chatbot

recipe_db = recipe_df[['name','ingredients','steps','description','submitted']]
recipe_db.head()

,name,ingredients,steps,description,submitted
0,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed ...","['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,2005-09-16
1,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'egg...","['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,2002-06-17
2,all in the kitchen chili,"['ground beef', 'yellow onions', 'diced tomato...","['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,2005-02-25
3,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...",['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...",2003-04-14
4,amish tomato ketchup for canning,"['tomato juice', 'apple cider vinegar', 'sugar...",['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,2002-10-25


### Step 3 draft the recipe search bot

Imagine the following conversation:

```
A : What should I cook now?
Bot: any ingredient you want to taste in the meal?
A : fish/pork/chicken/rice/curry/tomato (one or more ingredients)
Bot: okay. {number} recipes match your choice. Is there any ingredient you hate? I can filter them out.
A : Garlic/Kimchi/chili/beef (one or more ingredients)
Bot: I see. After filtering out those ingredients, {number} recipes remain. Do you want the first one, a random recipe or the most recent recipe?
A: a random one.
```

In [10]:
# test filtering of favourable ingredients
liked_ingredients = ['chicken','tomato','potato']
mask1= recipe_db['ingredients'].str.contains('chicken')
mask2 = recipe_db['ingredients'].str.contains('tomato')
mask3 = recipe_db['ingredients'].str.contains('potato')
filtered1 = recipe_db[mask1&mask2&mask3]

#the following also works
#recipe_db[eval("&".join(['mask1','mask2','mask3']))]
# recipe_db[mask1][mask2][mask3]

In [11]:
filtered1

,name,ingredients,steps,description,submitted
35,homemade vegetable soup from a can,"['low sodium chicken broth', 'diced tomatoes',...","['combine all ingredients in large pot', 'brin...",my friend just made this for dinner the other ...,2004-03-20
320,bacon cheeseburger and fries soup,"['bacon', 'lean ground beef', 'onion', 'celery...","['in a large skillet , cook bacon', 'drain', '...","my son asked for cheeseburger soup, and having...",2011-01-26
1358,30 minute bean soup for 2,"['margarine', 'celery rib', 'carrot', 'onions'...","['in a medium saucepan over medium-high heat ,...",cooking for 2 is such a challenge! we love dri...,2007-04-10
1524,5 can ww soup,"['new potatoes', 'carrots', 'zucchini with ita...",['put all the above ingredients into crockpot ...,"this is easy, yummy and is only 5 points for t...",2006-02-08
2285,absolute best chicken soup,"['water', 'whole chicken', 'chicken bouillon c...",['fill a 12 quart pot a little more than halfw...,this is a family recipe i stumbled upon throug...,2008-12-19
...,...,...,...,...,...
229889,yummy coconut curry chicken,"['boneless skinless chicken thighs', 'salt and...","['season chicken pieces with salt and pepper',...",an adapted recipe... tweaked to mine and the d...,2009-02-22
230182,yummy vegetable soup,"['oil', 'zucchini', 'potato', 'broccoli', 'cor...","['heat large pot and add oil , onion , pepper ...",this is very good soup and full of veggies. gr...,2004-02-18
230353,zesty 30 minute bean soup,"['olive oil', 'chorizo sausage', 'onion', 'chi...","['heat oil in a large pot over high heat', 'ad...","big flavors from a quick, hearty soup.",2007-02-24
231179,zucchini garlic chicken skillet,"['boneless skinless chicken breast halves', 'o...","['toss chicken , onions and garlic with the ol...",a very quick and easy dish for a middle-of-the...,2001-11-08


In [13]:
def recipe_bot_query(liked_ingredients):
    
    '''
    Input:
    ingredients -- a list
    '''
    #initialize a new df
    filtered = recipe_db
    for i,ing in enumerate(liked_ingredients):
        mask= filtered['ingredients'].str.contains(ing)
        filtered = filtered[mask]
    
    return filtered

matching_recipes_df = recipe_bot_query(liked_ingredients)
matching_recipes_df

,name,ingredients,steps,description,submitted
35,homemade vegetable soup from a can,"['low sodium chicken broth', 'diced tomatoes',...","['combine all ingredients in large pot', 'brin...",my friend just made this for dinner the other ...,2004-03-20
320,bacon cheeseburger and fries soup,"['bacon', 'lean ground beef', 'onion', 'celery...","['in a large skillet , cook bacon', 'drain', '...","my son asked for cheeseburger soup, and having...",2011-01-26
1358,30 minute bean soup for 2,"['margarine', 'celery rib', 'carrot', 'onions'...","['in a medium saucepan over medium-high heat ,...",cooking for 2 is such a challenge! we love dri...,2007-04-10
1524,5 can ww soup,"['new potatoes', 'carrots', 'zucchini with ita...",['put all the above ingredients into crockpot ...,"this is easy, yummy and is only 5 points for t...",2006-02-08
2285,absolute best chicken soup,"['water', 'whole chicken', 'chicken bouillon c...",['fill a 12 quart pot a little more than halfw...,this is a family recipe i stumbled upon throug...,2008-12-19
...,...,...,...,...,...
229889,yummy coconut curry chicken,"['boneless skinless chicken thighs', 'salt and...","['season chicken pieces with salt and pepper',...",an adapted recipe... tweaked to mine and the d...,2009-02-22
230182,yummy vegetable soup,"['oil', 'zucchini', 'potato', 'broccoli', 'cor...","['heat large pot and add oil , onion , pepper ...",this is very good soup and full of veggies. gr...,2004-02-18
230353,zesty 30 minute bean soup,"['olive oil', 'chorizo sausage', 'onion', 'chi...","['heat oil in a large pot over high heat', 'ad...","big flavors from a quick, hearty soup.",2007-02-24
231179,zucchini garlic chicken skillet,"['boneless skinless chicken breast halves', 'o...","['toss chicken , onions and garlic with the ol...",a very quick and easy dish for a middle-of-the...,2001-11-08


In [14]:
# test filtering of disliked ingredients
filtered = matching_recipes_df

disliked_ingredients = ['garlic']
for i,ing in enumerate(disliked_ingredients):
    masked= filtered['ingredients'].str.contains(ing)
    filtered = filtered.mask(masked).dropna()
    
filtered

,name,ingredients,steps,description,submitted
35,homemade vegetable soup from a can,"['low sodium chicken broth', 'diced tomatoes',...","['combine all ingredients in large pot', 'brin...",my friend just made this for dinner the other ...,2004-03-20
1358,30 minute bean soup for 2,"['margarine', 'celery rib', 'carrot', 'onions'...","['in a medium saucepan over medium-high heat ,...",cooking for 2 is such a challenge! we love dri...,2007-04-10
2643,adzuki bean stew,"['adzuki beans', 'leek', 'carrot', 'sweet pota...",['rinse the beans after draining them from the...,i found this on a diabetic website while searc...,2007-07-15
3015,alabama camp stew,"['pork', 'chicken', 'roast beef', 'potatoes', ...","['combine all meat in a large dutch oven', 'ad...",this recipe makes a very thick stew that's rea...,2002-10-29
4116,almost vegetarian vegetable soup,"['yellow onion', 'butter', 'chicken broth', 'p...","['in a heavy stock pot , melt butter and saute...","this is a quick, easy soup that uses ingredien...",2006-01-08
...,...,...,...,...,...
221542,vegetable and pasta soup,"['onion', 'mushroom', 'canola oil', 'chicken s...","['using a dutch oven , saute the onion and mus...","this is a variation of a recipe i already had,...",2008-01-28
225593,whatever is available soup,"['beef broth', 'water', 'potatoes', 'crushed t...","['in a large cooking pot or large saucepan , a...",this is a soup recipe that i made up in a hurr...,2005-02-03
228466,ww 4 points savory potato and ham chunks,"['lean ham', 'olive oil', 'potatoes', 'fat-fre...",['spray very large nonstick skillet with cooki...,from ww simply delicious - uses only one skill...,2007-08-19
229839,yummy cabbage soup,"['crushed tomatoes', 'tomato paste', 'water', ...","['using a large pot , combine all ingredients ...",this was something i threw together with lefto...,2005-03-10


In [15]:
def recipe_remove(matching_recipes_df,disliked_ingredients):
    filtered = matching_recipes_df
    for i,ing in enumerate(disliked_ingredients):
        masked= filtered['ingredients'].str.contains(ing)
        filtered = filtered.mask(masked).dropna()
    
    return filtered

cleaned_recipes = recipe_remove(matching_recipes_df,disliked_ingredients)
cleaned_recipes

,name,ingredients,steps,description,submitted
35,homemade vegetable soup from a can,"['low sodium chicken broth', 'diced tomatoes',...","['combine all ingredients in large pot', 'brin...",my friend just made this for dinner the other ...,2004-03-20
1358,30 minute bean soup for 2,"['margarine', 'celery rib', 'carrot', 'onions'...","['in a medium saucepan over medium-high heat ,...",cooking for 2 is such a challenge! we love dri...,2007-04-10
2643,adzuki bean stew,"['adzuki beans', 'leek', 'carrot', 'sweet pota...",['rinse the beans after draining them from the...,i found this on a diabetic website while searc...,2007-07-15
3015,alabama camp stew,"['pork', 'chicken', 'roast beef', 'potatoes', ...","['combine all meat in a large dutch oven', 'ad...",this recipe makes a very thick stew that's rea...,2002-10-29
4116,almost vegetarian vegetable soup,"['yellow onion', 'butter', 'chicken broth', 'p...","['in a heavy stock pot , melt butter and saute...","this is a quick, easy soup that uses ingredien...",2006-01-08
...,...,...,...,...,...
221542,vegetable and pasta soup,"['onion', 'mushroom', 'canola oil', 'chicken s...","['using a dutch oven , saute the onion and mus...","this is a variation of a recipe i already had,...",2008-01-28
225593,whatever is available soup,"['beef broth', 'water', 'potatoes', 'crushed t...","['in a large cooking pot or large saucepan , a...",this is a soup recipe that i made up in a hurr...,2005-02-03
228466,ww 4 points savory potato and ham chunks,"['lean ham', 'olive oil', 'potatoes', 'fat-fre...",['spray very large nonstick skillet with cooki...,from ww simply delicious - uses only one skill...,2007-08-19
229839,yummy cabbage soup,"['crushed tomatoes', 'tomato paste', 'water', ...","['using a large pot , combine all ingredients ...",this was something i threw together with lefto...,2005-03-10


In [ ]:
# sort recipes by order/random/recency

def rank_recipe(cleaned_recipes,sort_order):
    
    if  'random' in sort_order: #random order
        cleaned_recipes = cleaned_recipes.sample(frac=1)
    elif 'recent' in sort_order: #sort by recency
        cleaned_recipes = cleaned_recipes.sort_values("submitted",ascending = False)
    
    return cleaned_recipes

# test the else condition(sort_order is original order)
sorted_recipes = rank_recipe(cleaned_recipes,"first")
sorted_recipes

In [17]:
#return the top row
sorted_recipes['name'].iloc[0]

'homemade  vegetable soup from a can'

### Step 4 Implement the imagined conversation

Recall the imagined conversation

```
A : What should I cook now?
Bot: any ingredient you want to taste in the meal?
A : fish/pork/chicken/rice/curry/tomato (one or more ingredients)
Bot: okay. {number} recipes match your choice. Is there any ingredient you hate? I can filter them out.
A : Garlic/Kimchi/chili/beef (one or more ingredients)
Bot: I see. After filtering out those ingredients, {number} recipes remain. Do you want the first one, a random recipe or the most recent recipe?
A: a random one.
```

In [41]:
# need one more function

def next_one(try_or_not):
    if try_or_not == "yes":
        print("These are the steps of making this recipe:")
        recipe_row = sorted_recipes.iloc[0]
    else:
        print("This is the next recipe:")
        recipe_row = sorted_recipes.iloc[1]
        print(f"Recipe name: {recipe_row['name'].capitalize()}")
        
    for step in eval(recipe_row['steps']): #turn str back into a list
            print("- "+step)
            
next_one('no')

This is the next recipe:
Recipe name: Easy cheesy tomato potato chickpea soup
- saute onion in olive oil
- add chicken broth , diced potatoes , salt , pepper , bay leaves , and fennel seed
- boil gently about 15 minutes until potatoes are tender , but not falling apart
- add tomatoes and chickpeas
- bring back up to a simmer for about 5 minutes
- add evaporated milk and cheese
- stir over lowest heat just until cheese melts ~ do not overheat or mixture will curdle !
- remove bay leaves before serving
- serves 8-10


### Step 5 Run the Recipe chatbot!

In [42]:
print("Ask chatbot: What should I cook now?")
while True:
    liked_ingredients = input("Any ingredient you want to taste in the meal?").lower().strip(" ").split(",")
    matching_recipes_df = recipe_bot_query(liked_ingredients)
    print(f"Okay! {len(matching_recipes_df)} recipes match your choice.")
    disliked_ingredients = input("Is there any ingredient you hate? I can filter them out.").lower().strip(" ").split(",")
    cleaned_recipes = recipe_remove(matching_recipes_df,disliked_ingredients)
    print(f"I see. After filtering out those ingredients, {len(cleaned_recipes)} recipes remain.")
    sort_order = input("Do you want the first one, a random recipe or the most recent recipe?").lower()
    sorted_recipes = rank_recipe(cleaned_recipes, sort_order)
    
    ## extending the unfinished conversation
    print("Here is the first recipe: " + sorted_recipes['name'].iloc[0].capitalize())
    try_or_not =  input("Do you want to try it?").lower()
    next_one(try_or_not)
    break #done conversation


Ask chatbot: What should I cook now?
Any ingredient you want to taste in the meal?chicken, Tomato, potato
Okay! 157 recipes match your choice.
Is there any ingredient you hate? I can filter them out.Garlic!
I see. After filtering out those ingredients, 155 recipes remain.
Do you want the first one, a random recipe or the most recent recipe?random
Here is the first recipe: Curried chicken breast
Do you want to try it?yes
These are the steps of making this recipe:
- measure spices into a small bowl
- place garlic , ginger and onion in another bowl
- place skillet over medium heat , add oil and butter
- wait for the butter to melt
- add the spices
- stir and cook for a minute or two until the spices become fragrant
- add onion , ginger and garlic
- cook , stirring frequently , until softened , about 5 minutes
- cut the chicken in half crosswise , so you have 6 evenly sized pieces
- add the chicken and cook 3 minutes , flipping the chicken occasionally
- add potatoes , tomatoes with it's j